# Rossman Pharmaceutical Sales Prediction

## Imports

In [3]:
import numpy as np
import pandas as pd
import mlflow
import warnings
import pickle
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [4]:
# sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

In [5]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from file_handler import FileHandler

## Preprocessing

We have alreadyseen which features influence sales in the exploratory section. And we have used many of the columns for the 4 machine learning models used as seen in the scripts. But for LSTM only the date and sales columns will be used.

In [6]:
file_handler = FileHandler()
train_sales = file_handler.read_csv("../data/train.csv")

In [8]:
data = train_sales.groupby("Date").agg({"Sales": "mean"})
data

,Sales
Date,
2013-01-01,87.284560
2013-01-02,6233.030493
2013-01-03,5693.112108
2013-01-04,5954.218834
2013-01-05,5337.751570
...,...
2015-07-27,9602.952466
2015-07-28,8174.953363
2015-07-29,7623.284305


In [9]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data)
data['DataScaled'] = scaled_array


In [10]:
SIZE = len(data.DataScaled)
WINDOW_SIZE = 48
BATCH_SIZE = SIZE - WINDOW_SIZE * 2
EPOCHS = 200

In [11]:
DateTrain = data.index.values[0:BATCH_SIZE]
DateValid = data.index.values[BATCH_SIZE:]
XTrain = data.DataScaled.values[0:BATCH_SIZE].astype('float32')
XValid = data.DataScaled.values[BATCH_SIZE:].astype('float32')

# Obtain shapes for vectors of size (,1) for dates series

DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)


Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series in a stateless LSTM:  (96,)


In [12]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape


TensorShape([846, 1])

In [13]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset


<TensorSliceDataset shapes: (1,), types: tf.float32>

In [14]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)


In [15]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])


[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [16]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))


In [17]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))


In [18]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)


In [19]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset


In [20]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)


## Model Training

In [21]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')


In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 8)           320       
_________________________________________________________________
lstm_1 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 533
Trainable params: 533
Non-trainable params: 0
_________________________________________________________________


In [22]:
mlflow.set_experiment("LSTM")
mlflow.tensorflow.autolog()
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 0.4081 - val_loss: 0.3956
Epoch 2/200
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1/1 [==============================] - 0s 133ms/step - loss: 0.4073 - val_loss: 0.3943
Epoch 3/200
1/1 [==============================] - 0s 123ms/step - loss: 0.4066 - val_loss: 0.3929
Epoch 4/200
1/1 [==============================] - 0s 71ms/step - loss: 0.4059 - val_loss: 0.3916
Epoch 5/200
1/1 [==============================] - 0s 87ms/step - loss: 0.4052 - val_loss: 0.3903
Epoch 6/200
1/1 [==============================] - 0s 81ms/step - loss: 0.4045 - val_loss: 0.3890
Epoch 7/200
1/1 [==============================] - 0s 78ms/step - loss: 0.4039 - val_loss: 0.3877
Epoch 8/200
1/1 [==============================] - 0s 71ms/step - loss: 0.4032 - val_loss: 0.3864
Epoch 9/200
1/1 [==============================] - 0s 73ms/step - loss: 0.4026 - val_loss: 0.3852
Epoch 10/200
1/1 [============================

In [24]:
from time import gmtime, strftime

In [25]:
time = strftime("%Y-%m-%d-%H:%M:%S", gmtime())
model.save(f'../models/LSTM_sales {time}.pkl')

INFO:tensorflow:Assets written to: ../models/LSTM_sales 2021-08-01-17:22:28.pkl/assets
